In [1]:
import sys
import findspark
import pyspark
!pip install folium
import folium
findspark.init("C:\\spark-3.1.2-bin-hadoop3.2\\spark-3.1.2-bin-hadoop3.2")

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import col
from pyspark.sql.types import *
from pyspark.sql.functions import avg
from ipywidgets import widgets
import seaborn as sns
import matplotlib.pyplot as plt
spark = SparkSession.builder.getOrCreate()

In [3]:
#Creating schema's
schema_pubs = StructType([
    StructField("Food Standard Agency's ID", StringType(), nullable=True),
    StructField("name", StringType(), nullable=True),
    StructField("address", StringType(), nullable=True),
    StructField("postcode", StringType(), nullable=True),
    StructField("easting", DoubleType(), nullable=True),
    StructField("northing", DoubleType(), nullable=True),
    StructField("latitude", DoubleType(), nullable=False),
    StructField("longitude", DoubleType(), nullable=False),
    StructField("local_authority", StringType(), nullable=True)

])

In [4]:
# importing data 
### accidentData_df geeft meer info over het incident
accidentData_df = spark.read.format('csv').option('header',True).load('../../Desktop/Data mining/Accidents0514.csv')
### casualtiesData_df geeft meer info over het slachtoffer
casualtiesData_df = spark.read.format('csv').option('header', True).load('../../Desktop/Data mining/Casualties0514.csv')
pubsData_df = spark.read.format('csv').option('header',False).schema(schema_pubs).load('open_pubs.csv')

In [5]:
# Data exploratie
### Filter de pub data set
counterRowsBeforePubs=pubsData_df.count()
pubsData_df=pubsData_df.dropna(subset=["latitude","longitude"])
print(counterRowsBeforePubs-pubsData_df.count()," rijen zijn verwijderd omdat ze lege waarden hadden.")
### nutteloze kolommen zoals Pedestrian_Crossing,Special_Condition_at_site, Carriageway_Hazards,  want volgens Kaggle hebben bijna alle records dezelfde data dus veel gaat deze kolom ons niet opleveren alleen maar een langere run time. heb ook Location_Northing_OSGR, Location_Easting_OSGR, Local_Authority_(District), Local_Authority_(Highway)1st_Road_Class, 1st_Road_Number, 2nd_Road_Class, 2nd_Road_Number, Pedestrian_Crossing-Human_Control, Pedestrian_Crossing-Physical_Facilities
accidentData_df = accidentData_df.drop("Pedestrian_Crossing", "Special_Condition_at_site","Local_Authority_(Highway)", "Carriageway_Hazards","Location_Northing_OSGR", "Location_Easting_OSGR", "Local_Authority_(District)", "Local_Authority_(Highway)1st_Road_Class", "1st_Road_Number", "2nd_Road_Class", "2nd_Road_Number", "Pedestrian_Crossing-Human_Control", "Pedestrian_Crossing-Physical_Facilities")
### In casualtiesData heb ik ook een paar kolommen verwijderd => Pedestian_Location, Pedestrian_Movement, Bus_or_Coach_, Vehicle_Reference, Casualty_Reference, Age_Band_of_Casualty, Pedestrian_Road_Maintenance_Worker, Casualty_Home_Area_Type
casualtiesData_df = casualtiesData_df.drop("Pedestian_Location", "Pedestrian_Movement", "Bus_or_Coach_Passenger", "Vehicle_Reference", "Casualty_Reference", "Age_Band_of_Casualty", "Pedestrian_Road_Maintenance_Worker","Casualty_Home_Area_Type" )
print("Kolommen Vehicle_Reference en Casualty_Reference zijn zeer nuttige gegevens maar die waarden komen precies niet overeen met die tabel op Kaggle dus heb ze weggelaten. Kan dat ik verkeerd ben maar heb niets gevonden.")

146  rijen zijn verwijderd omdat ze lege waarden hadden.
Kolommen Vehicle_Reference en Casualty_Reference zijn zeer nuttige gegevens maar die waarden komen precies niet overeen met die tabel op Kaggle dus heb ze weggelaten. Kan dat ik verkeerd ben maar heb niets gevonden.


In [6]:
### Filter de accident data set
counterRowsBeforePubs=accidentData_df.count()
accidentData_df=accidentData_df.na.drop()
print(counterRowsBeforePubs-accidentData_df.count()," rijen zijn verwijderd omdat ze lege waarden hadden.")
### Filter de casaulties data
counterRowsBeforeCasaul=casualtiesData_df.count()
casualtiesData_df=casualtiesData_df.na.drop()
print(counterRowsBeforeCasaul-casualtiesData_df.count()," rijen zijn verwijderd omdat ze lege waarden hadden.")
accidentData_df.show(5, vertical=True)

120599  rijen zijn verwijderd omdat ze lege waarden hadden.
0  rijen zijn verwijderd omdat ze lege waarden hadden.
-RECORD 0----------------------------------------------------
 Accident_Index                              | 200501BS00001 
 Longitude                                   | -0.191170     
 Latitude                                    | 51.489096     
 Police_Force                                | 1             
 Accident_Severity                           | 2             
 Number_of_Vehicles                          | 1             
 Number_of_Casualties                        | 1             
 Date                                        | 04/01/2005    
 Day_of_Week                                 | 3             
 Time                                        | 17:42         
 1st_Road_Class                              | 3             
 Road_Type                                   | 6             
 Speed_limit                                 | 30            
 Junction_Detail 

In [7]:
### twee dataframes in één dataframe joinen m.b.v. ID(Accident_Index). Beide dataframes hebben evenveel records.
joined = accidentData_df.alias("A").join(casualtiesData_df.alias("B"),col("A.Accident_Index") == col("B.Accident_Index"),"inner").select(col("A.Accident_Index"), col("A.Longitude"),col("A.Latitude"), col("A.Police_Force"), col("A.Accident_Severity"), col("A.Number_of_Vehicles"), col("A.Number_of_Casualties"), col("A.Date"), col("A.Day_of_Week"),col("A.Time"), col("A.1st_Road_Class"), col("A.Road_Type"), col("A.Speed_limit"), col("A.Junction_Detail"), col("A.Junction_Control"), col("A.Light_Conditions"), col("A.Weather_Conditions"), col("A.Road_Surface_Conditions"), col("A.Special_Conditions_at_Site"), col("A.Urban_or_Rural_Area"), col("A.Did_Police_Officer_Attend_Scene_of_Accident"), col("A.LSOA_of_Accident_Location"), col("B.Casualty_Class"), col("B.Sex_of_Casualty"), col("B.Age_of_Casualty"), col("B.Casualty_Severity"), col("B.Pedestrian_Location"), col("B.Car_Passenger"), col("B.Casualty_Type")).toDF("Accident_Index", "Longitude","Latitude","Police_Force", "Accident_Severity","Number_of_Vehicles", "Number_of_Casualties", "Date", "Day_of_Week","Time","1st_Road_Class", "Road_Type", "Speed_limit","Junction_Detail", "Junction_Control","Light_Conditions", "Weather_Conditions","Road_Surface_Conditions","Special_Conditions_at_Site","Urban_or_Rural_Area", "Did_Police_Officer_Attend_Scene_of_Accident", "LSOA_of_Accident_Location","Casualty_Class","Sex_of_Casualty","Age_of_Casualty","Casualty_Severity","Pedestrian_Location","Car_Passenger","Casualty_Type")

In [8]:
### Soorten voertuigen
from collections import Counter
from pyspark.sql.functions import count

joined.groupby("Casualty_Type").count().show(22)
print("Ongeveer 83% van alle accidenten werden veroorzaakt in een auto (Casualty_Type 9). Andere voertuigen zijn vooral brommers onderverdeeld in groepen (50cc, 125cc, 500cc enzo) andere voertuigen zijn wel een beetje raar boerderij voertuigen? taxi? trambestuurders? zelfs paarden. Kans dat er iemand met een paard naar een pub gaat is vrij klein denk ik. Dus wat doen wij? enkel autos en brommers bijhouden en de rest verwijderen of gewoon zo laten?")

+-------------+-------+
|Casualty_Type|  count|
+-------------+-------+
|           11|  57485|
|            3|  63957|
|            8|  28020|
|           22|    331|
|           16|   1116|
|            0| 251398|
|           98|     50|
|            5|  80881|
|           18|    102|
|           17|   1007|
|           90|   9338|
|           19|  45911|
|           23|     17|
|           97|    193|
|            9|1270126|
|            1| 169654|
|           20|   5500|
|           10|   6767|
|            4|  22117|
|           21|  11037|
|            2|  33572|
+-------------+-------+

Ongeveer 83% van alle accidenten werden veroorzaakt in een auto (Casualty_Type 9). Andere voertuigen zijn vooral brommers onderverdeeld in groepen (50cc, 125cc, 500cc enzo) andere voertuigen zijn wel een beetje raar boerderij voertuigen? taxi? trambestuurders? zelfs paarden. Kans dat er iemand met een paard naar een pub gaat is vrij klein denk ik. Dus wat doen wij? enkel autos en brommers bijhoud

In [9]:
### Casualty Class
joined.groupby("Casualty_Class").count().show()
print("Casualty_Class geeft aan of de persoon in kwestie een passagier(2), bestuurder van een voertuig(1) of een voetganger(3) is. Aangezien wij opzoek zijn naar accidenten die werden veroorzaakt door dronken mensen, lijkt het mij onwaarschijnlijk dat passagiers van voertuigen de oorzaak kunnen zijn van een accident dus ik dacht misschien Casualty_Class(2) weglaten?")

+--------------+-------+
|Casualty_Class|  count|
+--------------+-------+
|             3| 251398|
|             1|1307284|
|             2| 499897|
+--------------+-------+

Casualty_Class geeft aan of de persoon in kwestie een passagier(2), bestuurder van een voertuig(1) of een voetganger(3) is. Aangezien wij opzoek zijn naar accidenten die werden veroorzaakt door dronken mensen, lijkt het mij onwaarschijnlijk dat passagiers van voertuigen de oorzaak kunnen zijn van een accident dus ik dacht misschien Casualty_Class(2) weglaten?


In [10]:
### Leeftijd
joined.groupby("Age_of_Casualty").count().show(121)
print("Er zitten hier records van kinderen 3, 8 jaar enzo. in die andere excel bestand die jij hebt gestuurd was alle info over 16+ mensen. Dus hier weeral al die kinderen verwijderen? Age_of_Casualty met waarde -1 zijn gevallen waar de leeftijd niet gekend is")

+---------------+-----+
|Age_of_Casualty|count|
+---------------+-----+
|             51|22497|
|              7|10372|
|             54|19253|
|             -1|45488|
|             15|21030|
|             11|18280|
|            101|    2|
|             29|38486|
|             69| 8114|
|             42|33251|
|             73| 6948|
|             87| 2511|
|             64|11110|
|              3| 7515|
|             30|45375|
|             34|33905|
|             59|15507|
|              8|11348|
|             28|41181|
|             22|54072|
|             85| 3727|
|             35|37741|
|             52|21720|
|             16|38820|
|              0| 1715|
|             71| 7175|
|             98|   71|
|             47|27522|
|             99|   70|
|             96|  145|
|             43|32014|
|              5| 8934|
|             31|35359|
|            100|    4|
|             18|72211|
|             70| 9663|
|             61|13479|
|             27|42096|
|             75

In [36]:
def sort_age(age):
    age = int(age)
    if 16 <= age < 25:
        return "16-24"
    elif 25<=age <35:
        return "25-34"
    elif 35<= age <45:
        return "35-44"
    elif 45<= age <60:
        return "45-60"
    elif 60<= age <120:
        return "+60"
    elif 0 <= age <16:
        return "Child"
    else:
        return "Unknown"
sort_age(24)
joined = joined.withColumn("Age_of_Casualty", col("Age_of_Casualty").cast("int"))
joined = joined.withColumnRenamed("1st_Road_Class", "First_Road_Class")
joined2 = joined.rdd.map(lambda x: (x.Accident_Index, x.Longitude, 
                                    x.Latitude, x.Police_Force, x.Accident_Severity, x.Number_of_Vehicles, 
                                    x.Number_of_Casualties, x.Date, x.Day_of_Week, x.Time, x.First_Road_Class, 
                                    x.Road_Type, x.Speed_limit, x.Junction_Detail, x.Junction_Control, x.Light_Conditions, 
                                    x.Weather_Conditions,
                                    x.Road_Surface_Conditions, x.Special_Conditions_at_Site,
                                    x.Urban_or_Rural_Area, x.Did_Police_Officer_Attend_Scene_of_Accident, x.LSOA_of_Accident_Location, x.Casualty_Class, x.Sex_of_Casualty, x.Age_of_Casualty, x.Casualty_Severity, x.Pedestrian_Location, x.Car_Passenger, x.Casualty_Type, sort_age(x.Age_of_Casualty)))

joined2 = joined2.toDF(["Accident_Index", "Longitude" , "Latitude" , "Police_Force" , "Accident_Severity" ,
       "Number_of_Vehicles" , "Number_of_Casualties" , "Date" , "Day_of_Week" , "Time" , "1st_Road_Class" , "Road_Type" , "Speed_limit" , "Junction_Detail" , "Junction_Control" , "Light_Conditions" , "Weather_Conditions" , "Road_Surface_Conditions" , "Special_Conditions_at_Site" , "Urban_or_Rural_Area" ,
       "Did_Police_Officer_Attend_Scene_of_Accident" , "LSOA_of_Accident_Location" , "Casualty_Class" , "Sex_of_Casualty" , 
       "Age_of_Casualty" , "Casualty_Severity" , "Pedestrian_Location" , "Car_Passenger" , "Casualty_Type" , "Age_Cat"])


In [38]:
joined2.show(20, vertical= True)

-RECORD 0----------------------------------------------------
 Accident_Index                              | 200501BS70192 
 Longitude                                   | -0.182199     
 Latitude                                    | 51.493723     
 Police_Force                                | 1             
 Accident_Severity                           | 3             
 Number_of_Vehicles                          | 2             
 Number_of_Casualties                        | 1             
 Date                                        | 04/05/2005    
 Day_of_Week                                 | 4             
 Time                                        | 16:10         
 1st_Road_Class                              | 5             
 Road_Type                                   | 6             
 Speed_limit                                 | 30            
 Junction_Detail                             | 0             
 Junction_Control                            | -1            
 Light_C

In [25]:
joined

DataFrame[Accident_Index: string, Longitude: string, Latitude: string, Police_Force: string, Accident_Severity: string, Number_of_Vehicles: string, Number_of_Casualties: string, Date: string, Day_of_Week: string, Time: string, First_Road_Class: string, Road_Type: string, Speed_limit: string, Junction_Detail: string, Junction_Control: string, Light_Conditions: string, Weather_Conditions: string, Road_Surface_Conditions: string, Special_Conditions_at_Site: string, Urban_or_Rural_Area: string, Did_Police_Officer_Attend_Scene_of_Accident: string, LSOA_of_Accident_Location: string, Casualty_Class: string, Sex_of_Casualty: string, Age_of_Casualty: int, Casualty_Severity: string, Pedestrian_Location: string, Car_Passenger: string, Casualty_Type: string]

In [ ]:
joined2.registerTempTable('joined2')
qr = sqlContext.sql("SELECT COUNT()")

In [41]:
### Aantal accidenten per leeftijdscategorie
joined2.groupby("Age_Cat").count().show()

+-------+------+
|Age_Cat| count|
+-------+------+
|    +60|224925|
|Unknown| 45488|
|  45-60|339903|
|  35-44|338598|
|  25-34|402662|
|  Child|192400|
|  16-24|514603|
+-------+------+



In [42]:
### Aantal accidenten per Leeftijdscategorie en ernstigheid van het incident
joined2.groupby("Age_Cat", "Accident_Severity").count().show()
print("Zal proberen om er een staafdiagram van te maken")

+-------+-----------------+------+
|Age_Cat|Accident_Severity| count|
+-------+-----------------+------+
|  45-60|                2| 48449|
|Unknown|                2|  4766|
|Unknown|                1|   296|
|  35-44|                3|289531|
|    +60|                2| 40833|
|  25-34|                3|346000|
|  45-60|                3|285286|
|Unknown|                3| 40426|
|    +60|                3|177093|
|  Child|                3|160162|
|  45-60|                1|  6168|
|  Child|                1|  2021|
|  25-34|                2| 50670|
|  16-24|                2| 75699|
|  16-24|                3|429373|
|  35-44|                2| 43970|
|  35-44|                1|  5097|
|  Child|                2| 30217|
|  16-24|                1|  9531|
|  25-34|                1|  5992|
+-------+-----------------+------+
only showing top 20 rows

